# Branch and bound

In this Notebook, we will talk about the branch and bound framework and it is based on Chapter 7 from the book Integer Programming (Wolsey, 1998).

*__Note__: the figures generated by this Notebook are better visualised if they are in a maximised window.*

The branch and bound framework is based on the algorithm design paradigm of divide and conquer. The key idea of this paradigm is trying to solve a problem by breaking it into subproblems which are easier to solve. In this Notebook, we will see how to apply this to integer programming problems.

First, we will import and define some functions we will use later.

In [1]:
import pulp
import networkx as nx  # We are using Networkx (https://networkx.github.io/)
import matplotlib.pyplot as plt
import numpy as np

%matplotlib

def drawTree(g, nodeLabels, edgeLabels, nodeColors = None):
    '''
    This function is used to draw a tree later.
    '''
    model = pulp.LpProblem('Tree drawing', sense=pulp.LpMinimize)

    h = 1.0
    x = pulp.LpVariable.dicts('x', g.nodes(), lowBound=0, cat=pulp.LpInteger)
    y = pulp.LpVariable.dicts('y', g.nodes(), lowBound=0, cat=pulp.LpInteger)

    model += sum(y[i] for i in g.nodes())

    for i, j in g.edges():
        model += x[j] == x[i] + 1.0
        if j % 2 == 0:  # Left child
            model += y[j] <= y[i] - h
        else:  # Right child
            model += y[j] >= y[i] + h

    
    for i in g.nodes():
        for j in g.nodes():
            if i < j:
                model += y[j] - y[i] >= 0.2 - (x[j] - x[i])*100
                
    for i in g.nodes():
        children = g.successors(i)
        if len(children) == 2:
            ai, bi = sorted(children)
            
            for j in g.nodes():
                auxC = g.successors(j)
                if len(auxC) == 2:
                    aj, bj = sorted(auxC)
                    model += y[bi] - y[ai] <= y[bj] - y[aj]

        elif len(children) == 1:
              model += y[i] - y[children[0]] <= 1.5
            
    assert model.solve() == pulp.LpStatusOptimal
    
    pos = {}
    for i in g.nodes():
        # print i, x[i].value(), y[i].value()
        pos[i] = (y[i].value(), -x[i].value())

        
    labels_pos = {}

    for n, p in pos.iteritems():
        if n % 2 == 1:
            labels_pos[n] = p[0] + 0.35, p[1]
        else:
            labels_pos[n] = p[0] - 0.35, p[1]

            
    if nodeColors == None:
        nx.draw_networkx(g, pos, node_color='m', node_size=1600)
    else:
        nx.draw_networkx(g, pos, node_color=nodeColors, node_size=1600)
        
    nx.draw_networkx_labels(g, labels_pos, labels=nodeLabels)
    nx.draw_networkx_edge_labels(g, pos, font_size=18, edge_labels=edgeLabels)

    xMin = np.Inf
    xMax = -np.Inf
    
    for x, _ in labels_pos.values():
        xMin = min(x, xMin)
        xMax = max(x, xMax)
        
    plt.axis('off')
    
    plt.xlim((xMin - 0.75, xMax + 0.75))
    plt.draw()
    plt.show()


def isInteger(a):
    '''
    This function is used to tell if a number is integer
    with precision epsilon.
    '''
    epsilon = 1e-5
    return a - int(a) < epsilon

Using matplotlib backend: TkAgg


## Complete enumeration

A strategy to solve a combinatorial problem is __complete enumeration__. In other words, we (try to) evaluate all possible solutions and choose one that gives the best value to the objective function. An usual way to implement this kind of strategy is by using an enumeration tree. We will see how this works for the following binary problem:

Max $3x_1 + 2x_2 + x_3 + 4x_4$ 

s. t.

$2x_1 + 3x_2 + 5x_3 + 3x_4 \leq 5$

$x_1, x_2, x_3, x_4, x_5 \in \{0, 1\}$.

In [5]:
nodeLabels = {}
edgeLabels = {}
tree = nx.DiGraph()

tree.add_edge(1, 2)
tree.add_edge(1, 3)
nodeLabels[1] = 0
nodeLabels[2] = 0
nodeLabels[3] = 3
edgeLabels[(1, 2)] = '$x_1 = 0$'
edgeLabels[(1, 3)] = '$x_1 = 1$'

tree.add_edge(2, 4)
tree.add_edge(2, 5)
nodeLabels[4] = 0
nodeLabels[5] = 2
edgeLabels[(2, 4)] = '$x_2 = 0$'
edgeLabels[(2, 5)] = '$x_2 = 1$'

tree.add_edge(3, 6)
tree.add_edge(3, 7)
nodeLabels[6] = 3
nodeLabels[7] = 5
edgeLabels[(3, 6)] = '$x_3 = 0$'
edgeLabels[(3, 7)] = '$x_3 = 1$'

tree.add_edge(4, 8)
tree.add_edge(4, 9)
nodeLabels[8] = 0
nodeLabels[9] = 1
edgeLabels[(4, 8)] = '$x_4 = 0$'
edgeLabels[(4, 9)] = '$x_4 = 1$'

drawTree(tree, nodeLabels, edgeLabels)

The root of the tree represents the original problem and each child node of the tree represents a subproblem. Each subproblem is obtained by choosing a value to a variable. Node number $3$, for instance, corresponds to the subproblems with $x_1 = 1$. This procedure can be applied recursively in order to enumerate all feasible solutions. In our example, the labels of the nodes represent the contribution of all the previously assigned variable to the objective function.

Note that we did not expanded all the tree. In fact, the potential number of nodes in a binary problem is proportional to $2^n$, which makes it impractical to expand all the search tree for problems that are of practical interest. Therefore, we need a smarter way to explore the feasible solution space.


## Implicit enumeration

Note that in each node we face two decisions: 

- which variable to constraint for further devoloping the tree? 
- Which node to explore next?

The answer to the first question is called __branching variable__ and the answer to the second question is the __branching node__. There are many ways of answering to these questions and will we just talk about them briefly latter. Now, we will see how we can use bounding information to reduce the number of nodes we need to explore.

We will some example. The first one is:

In [6]:
import networkx as nx

g = nx.DiGraph()
nodeLabels = {}

g.add_edge(1, 2)
g.add_edge(1, 3)
nodeLabels[1] = 'P = 10\nD= 20'
nodeLabels[2] = 'P = 16\nD= 20'
nodeLabels[3] = 'P = 12\nD= 15'

drawTree(g, nodeLabels, {})

Let the tree in the figure be a subtree during a tree search. The labels $P$ and $D$ refer to the value of primal and dual bounds, respectively, for the problem in the correspondent node. Also, we will assume it is a maximisation problem. For example, from node $1$, we know the solution we are searching is in $[10, 20]$. Since we have not found an optimal solution yet, we need to choose which node to further explore in the following step. The bounds from node $2$ tell us that we already have a feasible solution whose value is $16$. Also, the best we will have from further exploring node $2$ is a solution of value $20$. On the other hand, from node $3$ we have a feasible solution of value $12$ and the best we will get in further exploring is $16$. Note that we will gain nothing by further exploring node $3$ since we already have a solution (primal bound from node $2$) that is not worse than the best solution node $3$ can offer. In this situation we say that node $3$ is pruned *by bound*. We can also have a prune by *optimality* and by *infeasibility*.
Let us see another example.

In [7]:
import networkx as nx

g = nx.DiGraph()
nodeLabels = {}

g.add_edge(1, 2)
g.add_edge(1, 3)
nodeLabels[1] = 'P = 10\nD= 20'
nodeLabels[2] = 'P = 16\nD= 20'
nodeLabels[3] = 'P = 17\nD= 17'

drawTree(g, nodeLabels, {})

In the previous search tree, node $3$ have equal primal and dual bounds, which means we have an optimal solution to the node, thus, there is no need to further branching on node $3$. We say that node $3$ is pruned *by optimality*. Note that an analogous situation arises if during the search we found an infeasible problem. Then we say the correspoding node is pruned by *infeasiblity*. Let us see one more example:

In [8]:
import networkx as nx

g = nx.DiGraph()
nodeLabels = {}

g.add_edge(1, 2)
g.add_edge(1, 3)
nodeLabels[1] = 'P = 10\nD= 20'
nodeLabels[2] = 'P = 16\nD= 20'
nodeLabels[3] = 'P = 12\nD= 18'

drawTree(g, nodeLabels, {})

Note that, in this tree, none of the previous prunes are possible. In fact, it is not possible to prune any node and further exploration of the tree is needed. 

It is noteworthy that the branch and bound framework does __not__ require a mathematical model to be useful. The key of the framework is being able to find primal and dual bounds, thus, any algorithm able of finding such bounds can be used to build a branch a bound strategy. In mixed integer optimisation softwares the primal bounds are given by feasible solutions and the dual bounds are obtained using linear relaxation.

Let us build an example for the following integer program:

Min $-2x_1 - 3x_2$ 

s. t.

$x_1 + 2x_2 \leq 7$,

$2x_1 + x_2 \leq 7$,

$-x_1 + 4x_2 \leq 10$,

$12x_1 + 5x_2 \leq 30$,

$x_1, x_2 \in \mathbb{Z}_+$.

We will build the linear relaxation of the model using PuLP:

In [11]:
import pulp

models = {}
nodeLabels = {}
edgeLabels = {}

baseModel = pulp.LpProblem('BaseModel', pulp.LpMinimize)

# Note that we are defining the variables are continuous!
x1 = pulp.LpVariable('x1', lowBound=0, cat='Continuous')
x2 = pulp.LpVariable('x2', lowBound=0, cat='Continuous')

baseModel += -2*x1 - 3*x2  # Objective function

baseModel += x1 + 2*x2 <= 7
baseModel += 2*x1 + x2 <= 7
baseModel += -x1 + 4*x2 <= 10
baseModel += 12*x1 + 5*x2 <= 39

models[1] = baseModel

bbTree = nx.DiGraph()
bbTree.add_node(1)

# baseModel.solve()
# print baseModel.objective.value()
# print x1.value(), x2.value()

Let us start by exploring node $1$.

In [12]:
exploreNode = 1  # This is the node we are going to explore.

The following code solves the linear model, chooses a branching variable and calculates some of the bounds.

In [13]:
models[exploreNode].solve()

if models[exploreNode].status == pulp.constants.LpStatusOptimal:
    allInteger = True
    varValues = ''
    for v in models[exploreNode].variables():
        varValues += '%s = %.2f\n' % (v, v.value())
        if not isInteger(v.value()):
            allInteger = False
            lSon = 2*exploreNode
            rSon = 2*exploreNode + 1
            models[lSon] = models[exploreNode].deepcopy()
            models[lSon].name = lSon
            models[lSon] += v <= int(v.value())

            models[rSon] = models[exploreNode].deepcopy()
            models[rSon].name = rSon  
            models[rSon] += v >= int(v.value()) + 1

            bbTree.add_edge(exploreNode, lSon)
            edgeLabels[(exploreNode, lSon)] = r'$%s \leq %d$' % (v, int(v.value()))

            bbTree.add_edge(exploreNode, rSon)
            edgeLabels[(exploreNode, rSon)] = r'$%s \geq %d$' % (v, int(v.value()) + 1)

        if allInteger:  # We found a feasible solution!
            nodeLabels[exploreNode] = 'P = %.2f\n' % models[exploreNode].objective.value()
        else:
            nodeLabels[exploreNode] = 'D = %.2f\n' % models[exploreNode].objective.value()

        nodeLabels[exploreNode] += varValues
        models[exploreNode].status  
else:
    nodeLabels[exploreNode] = 'Model is\ninfeasible'

drawTree(bbTree, nodeLabels, edgeLabels)

Here, the labels in the side of each node show the value of the variables and the primal __or__ dual bound found by solving the problem associated to that node. You can now go back and choose another node to explore. You can try different rules for exploring the tree: breadth-first search, depth-first search and rules based on the bounds. Try to get a felling of how these rules can affect the number of nodes explored before find a feasible or optimal solution.